In [2]:
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from collections import Counter
import math as m

In [219]:
submissions_data=pd.read_csv(r'C:\\Users\\01\\Desktop\\GENERAL_ACCES\\submissions_data_train.csv')
events_data=pd.read_csv(r'C:\\Users\\01\\Desktop\\GENERAL_ACCES\\event_data_train.csv')


In [296]:
example=pd.read_csv(r'C:\\Users\\01\\Desktop\\GENERAL_ACCES\\submission_example.csv')

In [297]:
example

,user_id,is_gone
0,12,0.26
1,13,0.79
2,15,0.12
3,21,0.67
4,35,0.62
...,...,...
6179,7315,0.72
6180,24300,0.35
6181,820,0.88
6182,3649,0.38


In [164]:
event_data_train['day'] = event_data_train.date.dt.date

In [7]:
event_data_train

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15
...,...,...,...,...,...,...
3480698,33375,1526772685,viewed,11677,2018-05-19 23:31:25,2018-05-19
3480699,32815,1526772750,viewed,5198,2018-05-19 23:32:30,2018-05-19
3480700,32815,1526772750,discovered,5198,2018-05-19 23:32:30,2018-05-19
3480701,32815,1526772750,passed,5198,2018-05-19 23:32:30,2018-05-19


In [29]:
gap_data = event_data_train[['user_id', 'day', 'timestamp']].drop_duplicates(subset=['user_id', 'day'])\
    .groupby('user_id')['timestamp'].apply(list) \
    .apply(np.max).values

In [31]:
gap_data = pd.to_datetime(gap_data, unit = 's')

In [33]:
gap_data = gap_data.dt.date

In [34]:
gap_data

0        2016-09-02
1        2018-02-21
2        2015-10-11
3        2017-07-12
4        2018-03-21
            ...    
19229    2018-02-01
19230    2017-11-09
19231    2017-11-30
19232    2017-11-06
19233    2018-03-13
Length: 19234, dtype: object

In [30]:
gap_data = pd.Series(gap_data)

In [22]:
gap_data/(24*60*60)

0        17046.614167
1        17583.645440
2        16719.362280
3        17359.486354
4        17611.512269
             ...     
19229    17563.609410
19230    17479.456377
19231    17500.405544
19232    17476.774375
19233    17603.886146
Length: 19234, dtype: float64

In [42]:
rofl_df['last_day'] = gap_data

In [48]:
rofl_df['max_day'] = rofl_df['last_day'].max()

In [67]:
rofl_df['days_past']= rofl_df['max_day'] - rofl_df['last_day']

In [74]:
rofl_df['days_past'] = rofl_df['days_past'].dt.days

In [128]:
clear_train_df = rofl_df

In [89]:
clear_train_df = clear_train_df.loc[~((clear_train_df['days_past']<30) 
                                       & (15 < clear_train_df['passed']) 
                                       & (clear_train_df['passed']<40))]

In [155]:
clear_train_df

,user_id,viewed,discovered,started_attempt,passed,last_day,max_day,days_past,is_gone_user
0,1,1,1,0,0,2016-09-02,2018-05-19,624,True
1,2,10,9,2,9,2018-02-21,2018-05-19,87,True
2,3,192,91,30,87,2015-10-11,2018-05-19,951,True
3,5,12,11,4,11,2017-07-12,2018-05-19,311,True
4,7,1,1,0,1,2018-03-21,2018-05-19,59,True
...,...,...,...,...,...,...,...,...,...
19229,26790,9,8,1,8,2018-02-01,2018-05-19,107,True
19230,26793,1,1,1,0,2017-11-09,2018-05-19,191,True
19231,26794,180,69,34,69,2017-11-30,2018-05-19,170,True
19232,26797,13,10,2,10,2017-11-06,2018-05-19,194,True


In [102]:
sub_data_train.loc[sub_data_train['submission_status'] =='correct', 'correct'] = 1
sub_data_train.loc[sub_data_train['submission_status'] =='wrong', 'wrong'] = 1
sub_data_train = sub_data_train.fillna(0)

In [259]:
clear_train_df['is_gone_user'] = clear_train_df['days_past']>30

In [159]:
clear_train_df = clear_train_df.merge(users_scores, how='outer')

In [174]:
clear_train_df = clear_train_df.fillna(0)

In [152]:
users_scores = sub_data_train.pivot_table(index='user_id',
                                         columns='submission_status',
                                         values = 'step_id',
                                         aggfunc='count',
                                         fill_value=0).reset_index()

In [153]:
users_scores



submission_status,user_id,correct,wrong
0,2,2,0
1,3,29,23
2,5,2,2
3,8,9,21
4,14,0,1
...,...,...,...
9935,26787,3,0
9936,26790,1,0
9937,26794,33,9
9938,26797,2,0


In [169]:
users_days = event_data_train.groupby('user_id').day.nunique().to_frame().reset_index()

In [165]:
event_data_train.head()

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15


In [170]:
users_days

,user_id,day
0,1,1
1,2,2
2,3,7
3,5,2
4,7,1
...,...,...
19229,26790,4
19230,26793,1
19231,26794,9
19232,26797,2


In [172]:
clear_train_df = clear_train_df.merge(users_days, how='outer')

In [260]:
clear_train_df['passed_course'] = clear_train_df.correct>40

In [180]:
clear_train_df

,user_id,viewed,discovered,started_attempt,passed,last_day,max_day,days_past,is_gone_user,correct,wrong,day,passed_course
0,1,1,1,0,0,2016-09-02,2018-05-19,624,True,0.0,0.0,1,False
1,2,10,9,2,9,2018-02-21,2018-05-19,87,True,2.0,0.0,2,False
2,3,192,91,30,87,2015-10-11,2018-05-19,951,True,29.0,23.0,7,False
3,5,12,11,4,11,2017-07-12,2018-05-19,311,True,2.0,2.0,2,False
4,7,1,1,0,1,2018-03-21,2018-05-19,59,True,0.0,0.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19229,26790,9,8,1,8,2018-02-01,2018-05-19,107,True,1.0,0.0,4,False
19230,26793,1,1,1,0,2017-11-09,2018-05-19,191,True,0.0,0.0,1,False
19231,26794,180,69,34,69,2017-11-30,2018-05-19,170,True,33.0,9.0,9,False
19232,26797,13,10,2,10,2017-11-06,2018-05-19,194,True,2.0,0.0,2,False


In [183]:
clear_train_df[clear_train_df['passed_course']].day.median()

13.0

In [190]:
user_min_time = event_data_train.groupby('user_id', as_index=False)\
    .agg({'timestamp':'min'})\
    .rename({'timestamp':'min_timestamp'},axis=1)

In [192]:
clear_train_df = clear_train_df.merge(user_min_time,how='outer')

In [194]:
clear_train_df = clear_train_df.drop(['max_day'],axis=1)

In [195]:
events_data = event_data_train

In [199]:
event_data_train = events_data[events_data.user_id events_data.timestamp]

SyntaxError: invalid syntax (Temp/ipykernel_12252/2727259729.py, line 1)

In [200]:
events_data['user_time'] = events_data.user_id.map(str)+' '+events_data.timestamp.map(str)

In [201]:
events_data

,step_id,timestamp,action,user_id,date,day,user_time
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632 1434340895
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632 1434340895
...,...,...,...,...,...,...,...
3480698,33375,1526772685,viewed,11677,2018-05-19 23:31:25,2018-05-19,11677 1526772685
3480699,32815,1526772750,viewed,5198,2018-05-19 23:32:30,2018-05-19,5198 1526772750
3480700,32815,1526772750,discovered,5198,2018-05-19 23:32:30,2018-05-19,5198 1526772750
3480701,32815,1526772750,passed,5198,2018-05-19 23:32:30,2018-05-19,5198 1526772750


In [202]:
learning_time_threshold = 3*24*60*60

In [203]:
user_learning_time_threshold = user_min_time.user_id.map(str) + ' ' +(user_min_time.min_timestamp+learning_time_threshold).map(str)

In [204]:
user_learning_time_threshold

0            1 1473086664
1            2 1514642564
2            3 1434617676
3            5 1466416009
4            7 1521893860
               ...       
19229    26790 1451743216
19230    26793 1510484231
19231    26794 1495546249
19232    26797 1508684516
19233    26798 1521234963
Length: 19234, dtype: object

In [205]:
user_min_time['user_learning_time_threshold'] = user_learning_time_threshold

In [207]:
events_data = events_data.merge(user_min_time[['user_id', 'user_learning_time_threshold']],how ='outer')

In [208]:
event_data_train = events_data[events_data.user_time <= events_data.user_learning_time_threshold]

In [210]:
event_data_train.groupby('user_id').day.nunique().max()

4

In [323]:
submissions_data['user_time'] = submissions_data.user_id.map(str) + ' ' +submissions_data.timestamp.map(str)
submissions_data = submissions_data.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how = 'outer')
submissions_data_train = submissions_data[submissions_data.user_time <= submissions_data.user_learning_time_threshold]


In [311]:
submissions_data['date'] = pd.to_datetime(submissions_data.timestamp, unit = 's')

In [312]:
submissions_data['day'] = submissions_data.date.dt.date

In [426]:
X = submissions_data_train.groupby('user_id').day.nunique().to_frame().reset_index()\
    .rename(columns={'day':'days'})

In [325]:
steps_tried = submissions_data_train.groupby('user_id').step_id.nunique().to_frame().reset_index()\
    .rename(columns={'step_id': 'steps_tried'})

In [427]:
X=X.merge(steps_tried, on='user_id', how = 'outer')

In [327]:
X

,user_id,days,steps_tried
0,2,1,2
1,3,1,4
2,8,1,11
3,14,1,1
4,16,3,23
...,...,...,...
8908,26787,1,3
8909,26790,1,1
8910,26794,3,27
8911,26797,1,2


In [428]:
X = X.merge(submissions_data_train.pivot_table(index='user_id',
                                         columns='submission_status',
                                         values = 'step_id',
                                         aggfunc='count',
                                         fill_value=0).reset_index())

In [329]:
X

,user_id,days,steps_tried,correct,wrong
0,2,1,2,2,0
1,3,1,4,4,4
2,8,1,11,9,21
3,14,1,1,0,1
4,16,3,23,23,27
...,...,...,...,...,...
8908,26787,1,3,3,0
8909,26790,1,1,1,0
8910,26794,3,27,31,9
8911,26797,1,2,2,0


In [429]:
X['correct_ratio'] = X.correct/(X.correct+X.wrong)

In [430]:
X = X.merge(event_data_train.pivot_table(index='user_id',
                             columns='action',
                             values='step_id',
                             aggfunc='count',
                             fill_value=0).reset_index()[['user_id', 'viewed']], how='outer')

In [431]:
X=X.fillna(0)

In [432]:
X = X.merge(clear_train_df[['user_id', 'passed_course', 'is_gone_user']], how='outer')

In [433]:
X = X[~((X.is_gone_user == False)&(X.passed_course == False))]

In [434]:
X

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_course,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True
...,...,...,...,...,...,...,...,...,...
19229,26774,0.0,0.0,0.0,0.0,0.00,1,False,True
19230,26781,0.0,0.0,0.0,0.0,0.00,6,True,True
19231,26788,0.0,0.0,0.0,0.0,0.00,1,False,True
19232,26789,0.0,0.0,0.0,0.0,0.00,2,False,True


In [435]:
X.groupby(['passed_course', 'is_gone_user']).user_id.count()

passed_course  is_gone_user
False          True            16074
True           False             226
               True             1742
Name: user_id, dtype: int64

In [436]:
y=X.passed_course

In [440]:
X_train = X.drop(['passed_course','is_gone_user','user_id'],axis=1)

In [441]:
X_train

,days,steps_tried,correct,wrong,correct_ratio,viewed
0,1.0,2.0,2.0,0.0,1.00,9
1,1.0,4.0,4.0,4.0,0.50,20
2,1.0,11.0,9.0,21.0,0.30,154
3,1.0,1.0,0.0,1.0,0.00,9
4,3.0,23.0,23.0,27.0,0.46,132
...,...,...,...,...,...,...
19229,0.0,0.0,0.0,0.0,0.00,1
19230,0.0,0.0,0.0,0.0,0.00,6
19231,0.0,0.0,0.0,0.0,0.00,1
19232,0.0,0.0,0.0,0.0,0.00,2


In [ ]:
ОБУЧЕНИЕ




In [277]:
clf = RandomForestClassifier()

In [447]:
params = {
    'n_estimators':range(10,51,10),
'max_depth':range(1,13,2),
'min_samples_leaf':range(1,8),
'min_samples_split':range(2,10,2)   
}

search = GridSearchCV(clf, params, n_jobs=-1)

In [448]:
search.fit(X_train,y)

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 10, 2),
                         'n_estimators': range(10, 51, 10)})

In [449]:
search.best_params_

{'max_depth': 9,
 'min_samples_leaf': 6,
 'min_samples_split': 4,
 'n_estimators': 20}

In [450]:
best_forest = search.estimator

In [451]:
best_forest.fit(X_train,y)

RandomForestClassifier()

In [462]:
predictions = best_forest.predict_proba(X_test1)[:,1]

In [407]:
predict_df = predict_df.drop(['days','steps_tried','correct','wrong','correct_ratio','viewed'], axis=1)

In [463]:
predict_df['is_gone'] = predictions

In [464]:
predict_df

,user_id,is_gone
0,12,0.172194
1,13,0.290000
2,15,0.110000
3,21,0.350000
4,35,0.060000
...,...,...
6179,26745,0.013973
6180,26768,0.013973
6181,26791,0.013973
6182,26795,0.013973


In [465]:
predict_df.to_csv(r'C:\\Users\\01\\Desktop\\GENERAL_ACCES\\PREDICTIONS.csv')

In [412]:
submissions_data_test=pd.read_csv(r'C:\\Users\\01\\Desktop\\GENERAL_ACCES\\submission_data_test.csv')
events_data_test=pd.read_csv(r'C:\\Users\\01\\Desktop\\GENERAL_ACCES\\events_data_test.csv')

In [413]:
users_scores_test = submissions_data_test.pivot_table(index='user_id',
                                         columns='submission_status',
                                         values = 'step_id',
                                         aggfunc='count',
                                         fill_value=0).reset_index()

In [308]:
users_scores_test

submission_status,user_id,correct,wrong
0,12,1,0
1,13,29,36
2,15,10,30
3,21,24,103
4,35,7,35
...,...,...,...
2798,26775,46,160
2799,26780,16,7
2800,26785,3,1
2801,26796,2,3


In [414]:
X_test = pd.DataFrame()

In [415]:
submissions_data_test['date'] = pd.to_datetime(submissions_data_test.timestamp, unit = 's')
submissions_data_test['day'] = submissions_data_test.date.dt.date

In [416]:
X_test = submissions_data_test.groupby('user_id').day.nunique().to_frame().reset_index()\
    .rename(columns={'day':'days'})

In [417]:
steps_tried_test = submissions_data_test.groupby('user_id').step_id.nunique().to_frame().reset_index()\
    .rename(columns={'step_id': 'steps_tried'})

In [418]:
X_test=X_test.merge(steps_tried_test, on='user_id', how = 'outer')

In [419]:
X_test = X_test.merge(submissions_data_test.pivot_table(index='user_id',
                                         columns='submission_status',
                                         values = 'step_id',
                                         aggfunc='count',
                                         fill_value=0).reset_index())

In [420]:
X_test

,user_id,days,steps_tried,correct,wrong
0,12,1,1,1,0
1,13,2,29,29,36
2,15,3,11,10,30
3,21,1,30,24,103
4,35,3,9,7,35
...,...,...,...,...,...
2798,26775,2,38,46,160
2799,26780,3,16,16,7
2800,26785,1,3,3,1
2801,26796,1,2,2,3


In [421]:
X_test['correct_ratio'] = X_test.correct/(X_test.correct+X_test.wrong)

In [422]:
X_test = X_test.merge(event_data_test.pivot_table(index='user_id',
                             columns='action',
                             values='step_id',
                             aggfunc='count',
                             fill_value=0).reset_index()[['user_id', 'viewed']], how='outer')

In [423]:
X_test = X_test.fillna(0)

In [453]:
X_test1 = X_test.drop('user_id', axis=1)

In [454]:
X_test1

,days,steps_tried,correct,wrong,correct_ratio,viewed
0,1.0,1.0,1.0,0.0,1.000000,14
1,2.0,29.0,29.0,36.0,0.446154,105
2,3.0,11.0,10.0,30.0,0.250000,1
3,1.0,30.0,24.0,103.0,0.188976,98
4,3.0,9.0,7.0,35.0,0.166667,70
...,...,...,...,...,...,...
6179,0.0,0.0,0.0,0.0,0.000000,1
6180,0.0,0.0,0.0,0.0,0.000000,1
6181,0.0,0.0,0.0,0.0,0.000000,1
6182,0.0,0.0,0.0,0.0,0.000000,1
